In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error


In [ ]:
!pip install autoviz
!pip install xlrd

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class
AV = AutoViz_Class()

In [ ]:
ff = AV.AutoViz('../input/30-days-of-ml/train.csv')

In [ ]:
df_train = pd.read_csv('../input/5-folds/train_5_folds.csv')
df_test = pd.read_csv('../input/30-days-of-ml/test.csv')
sample_submission = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')



In [ ]:
features = [col for col in df_train.columns if col not in ("id", 'target', 'kfold')]

In [ ]:
object_cols = [col for col in features if 'cat' in col]

In [ ]:
df_test = df_test[features]

# KFold
created same as abhiske takur

In [ ]:
from xgboost import XGBRegressor
from xgboost import plot_tree
xgb_params = {
    'n_estimators': 1000,
    'learning_rate': 0.15,
    'subsample': 0.95,
    'colsample_bytree': 0.10,
    'max_depth': 3,
    'booster': 'gbtree',
    'reg_lambda': 66.1,
    'reg_alpha': 15.9,
    'random_state': 42
}

In [ ]:
final_predictions = []
for fold in range(5):
    xtrain =  df_train[df_train.kfold != fold].reset_index(drop=True)
    xvalid = df_train[df_train.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    ordinal_encoder = OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = XGBRegressor(**xgb_params)
    model.fit(xtrain, ytrain)
    
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    print(f"fold: {fold}, rmse: {mean_squared_error(yvalid, preds_valid, squared=False)}")

In [ ]:
preds = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.target = preds
sample_submission.to_csv("submission.csv", index=False)